In [1]:
from collections import defaultdict
from typing import List

import pandas as pd

In [2]:
def read_lines(file_path: str) -> List[str]:
    res = list()
    with open(file_path, 'r') as in_file:
        line = in_file.readline()
        while line:
            res.append(line)
            line = in_file.readline()
    return res

In [3]:
cpp_lines = read_lines('./cpp_benchmark.txt')
rust_lines = read_lines('./rust_benchmark.txt')
python_lines = read_lines('./python_benchmark.txt')

In [4]:
dicts = {
    'trapezoidal': {
        'cpp': defaultdict(dict),
        'rust': defaultdict(dict),
        'python': defaultdict(dict)
    },
    'simpson_1_3': {
        'cpp': defaultdict(dict),
        'rust': defaultdict(dict),
        'python': defaultdict(dict)
    },
    'simpson_3_8': {
        'cpp': defaultdict(dict),
        'rust': defaultdict(dict),
        'python': defaultdict(dict)
    }
}

In [5]:
for line in cpp_lines:
    cur_dict = None
    if 'Trapezoidal_f64' in line:
        cur_dict = dicts['trapezoidal']['cpp']
    elif 'Simpson13_f64' in line:
        cur_dict = dicts['simpson_1_3']['cpp']
    elif 'Simpson38_f64' in line:
        cur_dict = dicts['simpson_3_8']['cpp']
    if cur_dict is not None:
        line_sections = line.split()
        num_section = int(line_sections[0][line_sections[0].find('/') + 1:])
        duration = float(line_sections[1].split()[0])
        cur_dict[num_section] = duration

In [6]:
for line in rust_lines:
    cur_dict = None
    start_idx = None
    if 'time:' not in line:
        continue
    if 'Trapezoidal' in line:
        cur_dict = dicts['trapezoidal']['rust']
        start_idx = len('f64/Trapezoidal/')
    elif 'Simpson\'s 1/3' in line:
        cur_dict = dicts['simpson_1_3']['rust']
        start_idx = len('f64/Simpson\'s 1/3/')
    elif 'Simpson\'s 3/8' in line:
        cur_dict = dicts['simpson_3_8']['rust']
        start_idx = len('f64/Simpson\'s 3/8/')
    if cur_dict is not None:
        num_section = int(line[start_idx:start_idx + 3].strip())
        duration = float(line[line.find('[') + 1:].split()[0].strip())
        cur_dict[num_section] = duration

In [7]:
for line in python_lines:
    cur_dict = None
    start_idx = len('Trapezoidal ')
    if 'Trapezoidal' in line:
        cur_dict = dicts['trapezoidal']['python']
    elif 'Simpson 1/3' in line:
        cur_dict = dicts['simpson_1_3']['python']
    elif 'Simpson 3/8' in line:
        cur_dict = dicts['simpson_3_8']['python']
    if cur_dict is not None:
        num_section = int(line[start_idx:line.find(':')])
        trailing_str = line[line.find(':') + 1:]
        duration = float(trailing_str[:trailing_str.find('n')].strip())
        cur_dict[num_section] = duration

In [8]:
for method in ('trapezoidal', 'simpson_1_3', 'simpson_3_8'):
    cur_dict = dicts[method]
    data = {key: [ele[1] for ele in sorted(value.items(), key=lambda x: x[0])] for key, value in cur_dict.items()}
    df = pd.DataFrame(data=data)
    df.to_csv('./{0}.csv'.format(method))